# Load dataset

In [1]:
%%capture
!pip install mediapipe==0.9.0.1
!pip install protobuf==3.20.*
!pip install scikit-image

In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import tensorflow as tf
import json
import mediapipe
import matplotlib
import matplotlib.pyplot as plt
import random

from skimage.transform import resize
from mediapipe.framework.formats import landmark_pb2
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm
from matplotlib import animation, rc

In [3]:
random.seed(42)

In [4]:
print("TensorFlow v" + tf.__version__)
print("Mediapipe v" + mediapipe.__version__)

TensorFlow v2.10.1
Mediapipe v0.9.0.1


In [5]:
dataset_df = pd.read_csv('kaggle_dataset/train.csv')
print("Full train dataset shape is {}".format(dataset_df.shape))

Full train dataset shape is (94477, 4)


In [6]:
dataset_df.head()

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [17]:
dataset_df.value_counts("sign")

sign
listen    415
look      414
shhh      411
donkey    410
mouse     408
         ... 
person    312
dance     312
beside    310
vacuum    307
zipper    299
Name: count, Length: 250, dtype: int64

In [18]:
signs = sorted(set(dataset_df.sign))
signs_to_idx = {sign : i for i, sign in enumerate(signs)}
idx_to_sign = {i : sign for sign, i in signs_to_idx.items()}

In [20]:
# Fetch sequence_id, file_id, phrase from first row
path, sequence_id, sign = dataset_df.sample(n=1).iloc[0][['path', 'sequence_id', 'sign']]
print(f"sequence_id: {sequence_id}, path: {path}, sign: {sign}")

# Fetch data from parquet file (This dataset stores each landmark in a separate row)
df = pq.read_table(f"kaggle_dataset/{path}",).to_pandas()
print("Full sequence dataset shape is {}".format(df.shape))

sequence_id: 2827210123, path: train_landmark_files/62590/2827210123.parquet, sign: cloud
Full sequence dataset shape is (9774, 7)


# Sample

In [25]:
# Transform the dataframe to be tha same format as the fingerspelling dataset (1 row for each frame)

with open("name_to_idx.json", "r") as json_file:
    name_to_idx = json.load(json_file)

def transform_df(df):
    new_rows = []
    
    # Sort the values by frame so that we can create the new row for each frame
    df = df.sort_values("frame")
    
    # Create new row for the first frame (NaN for default value)
    new_row = [np.nan]*len(name_to_idx)
    new_row[0] = df["frame"][0]
    for i, row in df.iterrows():
        # If we switch reach a new frame we save the old one and initialize a new empty one
        if new_row[0] != row["frame"]:
            new_rows.append(new_row)
            new_row = [np.nan]*len(name_to_idx)
            new_row[0] = row["frame"]
        
        new_row[name_to_idx[f"x_{row['type']}_{row['landmark_index']}"]] = row['x']
        new_row[name_to_idx[f"y_{row['type']}_{row['landmark_index']}"]] = row['y']
        new_row[name_to_idx[f"z_{row['type']}_{row['landmark_index']}"]] = row['z']
        
    return pd.DataFrame(new_rows, columns=name_to_idx.keys())

In [26]:
def random_sequence():
    # Fetch sequence_id, file_id, phrase from first row
    path, sequence_id, sign = dataset_df.sample(n=1).iloc[0][['path', 'sequence_id', 'sign']]
    print(f"sequence_id: {sequence_id}, path: {path}, sign: {sign}")
    
    # Fetch data from parquet file (This dataset stores each landmark in a separate row)
    sample_sequence_df_wrong_format = pq.read_table(f"kaggle_dataset/{path}",).to_pandas()
    print("Full sequence dataset shape is {}".format(sample_sequence_df_wrong_format.shape))
    
    return sample_sequence_df_wrong_format

In [27]:
sample_sequence_df_wrong_format = random_sequence()

sequence_id: 1749168688, path: train_landmark_files/34503/1749168688.parquet, sign: shoe
Full sequence dataset shape is (15747, 7)


In [28]:
%%time
sample_sequence_df = transform_df(sample_sequence_df_wrong_format)

CPU times: total: 828 ms
Wall time: 831 ms


In [29]:
sample_sequence_df.head()

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,61,0.399437,0.412454,0.404690,0.396396,0.414637,0.414517,0.411284,0.266369,0.411855,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62,0.399627,0.412420,0.404657,0.396310,0.414588,0.414451,0.411157,0.266858,0.411642,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63,0.399531,0.412322,0.404580,0.396294,0.414505,0.414397,0.411164,0.266417,0.411668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64,0.402818,0.416515,0.408343,0.400190,0.418785,0.418590,0.415036,0.268264,0.415507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,65,0.402316,0.414731,0.406771,0.398685,0.417009,0.416932,0.413754,0.268655,0.414374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Animation preview

In [30]:
matplotlib.rcParams['animation.embed_limit'] = 2**128
matplotlib.rcParams['savefig.pad_inches'] = 0
rc('animation', html='jshtml')

def create_animation(images):
    fig = plt.figure(figsize=(6, 9))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    im=ax.imshow(images[0], cmap="gray")
    plt.close(fig)
    
    def animate_func(i):
        im.set_array(images[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames=len(images), interval=1000/10)

mp_pose = mediapipe.solutions.pose
mp_hands = mediapipe.solutions.hands
mp_holistic = mediapipe.solutions.holistic
mp_drawing = mediapipe.solutions.drawing_utils 
mp_drawing_styles = mediapipe.solutions.drawing_styles

def collect_landmarks_oftype(_type: str, frame_data: pd.Series):
    x_type = frame_data.filter(regex=f"x_{_type}.*").values
    y_type = frame_data.filter(regex=f"y_{_type}.*").values
    z_type = frame_data.filter(regex=f"z_{_type}.*").values
    landmarks = landmark_pb2.NormalizedLandmarkList()
    for x, y, z in zip(x_type, y_type, z_type):
        landmarks.landmark.add(x=x, y=y, z=z)
    return landmarks
        
def produce_video_from_seq(seq_df: pd.DataFrame):
    frames = []
    for seq_idx in range(len(seq_df)):
        frame = np.zeros((600, 600, 3))
        frame_data = seq_df.iloc[seq_idx]
        
        # Right hand
        right_hand_landmarks = collect_landmarks_oftype("right_hand", frame_data)
        mp_drawing.draw_landmarks(
                frame,
                right_hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style())
        
        # Left Hand
        left_hand_landmarks = collect_landmarks_oftype("left_hand", frame_data)
        mp_drawing.draw_landmarks(
                frame,
                left_hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style())
        
        # Face
        face_landmarks = collect_landmarks_oftype("face", frame_data)
        mp_drawing.draw_landmarks(
            frame,
            face_landmarks,
            mp_holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        
        # Pose
        pose_landmarks = collect_landmarks_oftype("pose", frame_data)
        mp_drawing.draw_landmarks(
            frame,
            pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.
            get_default_pose_landmarks_style())
        
        frames.append(frame.astype(np.uint8))
    return frames

In [60]:
rand_df = random_sequence()
video_df = transform_df(rand_df)
video = produce_video_from_seq(video_df)
create_animation(video)

sequence_id: 3597735922, path: train_landmark_files/53618/3597735922.parquet, sign: empty
Full sequence dataset shape is (9774, 7)


# Init feature vector

In [17]:
# Pose coordinates for hand movement.
LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

X = [f'x_right_hand_{i}' for i in range(21)] + [f'x_left_hand_{i}' for i in range(21)] + [f'x_pose_{i}' for i in POSE] + [f'x_face_{i}' for i in range(468)]
Y = [f'y_right_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] + [f'y_pose_{i}' for i in POSE] + [f'y_face_{i}' for i in range(468)]
Z = [f'z_right_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)] + [f'z_pose_{i}' for i in POSE] + [f'z_face_{i}' for i in range(468)]

FEATURE_COLUMNS = X + Y + Z

X_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "x_" in col]
Y_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "y_" in col]
Z_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "z_" in col]

RHAND_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "right" in col]
LHAND_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "left" in col]
RPOSE_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "pose" in col and int(col[-2:]) in RPOSE]
LPOSE_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "pose" in col and int(col[-2:]) in LPOSE]

# Save to TFRecords

In [18]:
# Takes around 15 hours
def save_tfrecords_to(folder: str):
    # Loop through each file_id
    for _, dataset_df_row in tqdm(dataset_df.iterrows(), total=len(dataset_df)):
        pq_file = dataset_df_row["path"]
        
        # Fetch the parquet file
        parquet_df = transform_df(pq.read_table(f"kaggle_dataset/{pq_file}",).to_pandas())

        # File name for the updated data
        tf_file = f"{folder}/{dataset_df_row['participant_id']}_{dataset_df_row['sequence_id']}.tfrecord"
        parquet_numpy = parquet_df.to_numpy()
        # Initialize the pointer to write the output of each `for loop` below as a sequence into the file.
        with tf.io.TFRecordWriter(tf_file) as file_writer:
            frames = parquet_df.to_numpy()
            phrase = dataset_df_row["sign"]
            features = {FEATURE_COLUMNS[i]: tf.train.Feature(float_list=tf.train.FloatList(value=frames[:, i])) for i in range(len(FEATURE_COLUMNS))}
            features["phrase"] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(phrase, 'utf-8')]))
            record_bytes = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()
            file_writer.write(record_bytes)

folder_name = "preprocessed"
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
    save_tfrecords_to(folder_name)
else:
    print("Warning: Tfrecords already exist! Delete the folder to regenerate them!")

# Fetch from TFRecords

In [19]:
tf_records = []
for _, dataset_df_row in dataset_df.iterrows():
    tf_records.append(f"{folder_name}/{dataset_df_row['participant_id']}_{dataset_df_row['sequence_id']}.tfrecord")
print(f"List of {len(tf_records)} TFRecord files.")

List of 94477 TFRecord files.


In [22]:
def decode_fn(record_bytes):
    schema = {COL: tf.io.VarLenFeature(dtype=tf.float32) for COL in FEATURE_COLUMNS}
    schema["phrase"] = tf.io.FixedLenFeature([], dtype=tf.string)
    features = tf.io.parse_single_example(record_bytes, schema)
    phrase = features["phrase"]
    landmarks = ([tf.sparse.to_dense(features[COL]) for COL in FEATURE_COLUMNS])
    # Transpose to maintain the original shape of landmarks data.
    landmarks = tf.transpose(landmarks)
    
    return landmarks, phrase

In [136]:
#FRAME_LEN = 100

# def preprocess_landmarks(x):
#     # Pad end of sequence to be exactly 50 frames long
#     if tf.shape(x)[0] < FRAME_LEN:
#         x = tf.pad(x, ([[0, FRAME_LEN-tf.shape(x)[0]], [0, 0]]))
#     else:
#         x = x[:FRAME_LEN, :]
        
#     # Replace NaN values with 0
#     x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
    
#     return x

FRAME_LEN = 128

# Function to resize and add padding.
def resize_pad(x):
    if tf.shape(x)[0] < FRAME_LEN:
        x = tf.pad(x, ([[0, FRAME_LEN-tf.shape(x)[0]], [0, 0], [0, 0]]))
    else:
        x = tf.image.resize(x, (FRAME_LEN, tf.shape(x)[1]))
    return x

# Detect the dominant hand from the number of NaN values.
# Dominant hand will have less NaN values since it is in frame moving.
def pre_process(x):
    rhand = tf.gather(x, RHAND_IDX, axis=1)
    lhand = tf.gather(x, LHAND_IDX, axis=1)
    rpose = tf.gather(x, RPOSE_IDX, axis=1)
    lpose = tf.gather(x, LPOSE_IDX, axis=1)
    
    rnan_idx = tf.reduce_any(tf.math.is_nan(rhand), axis=1)
    lnan_idx = tf.reduce_any(tf.math.is_nan(lhand), axis=1)
    
    rnans = tf.math.count_nonzero(rnan_idx)
    lnans = tf.math.count_nonzero(lnan_idx)
    
    # For dominant hand
    if rnans > lnans:
        hand = lhand
        pose = lpose
        
        hand_x = hand[:, 0*(len(LHAND_IDX)//3) : 1*(len(LHAND_IDX)//3)]
        hand_y = hand[:, 1*(len(LHAND_IDX)//3) : 2*(len(LHAND_IDX)//3)]
        hand_z = hand[:, 2*(len(LHAND_IDX)//3) : 3*(len(LHAND_IDX)//3)]
        hand = tf.concat([1-hand_x, hand_y, hand_z], axis=1)
        
        pose_x = pose[:, 0*(len(LPOSE_IDX)//3) : 1*(len(LPOSE_IDX)//3)]
        pose_y = pose[:, 1*(len(LPOSE_IDX)//3) : 2*(len(LPOSE_IDX)//3)]
        pose_z = pose[:, 2*(len(LPOSE_IDX)//3) : 3*(len(LPOSE_IDX)//3)]
        pose = tf.concat([1-pose_x, pose_y, pose_z], axis=1)
    else:
        hand = rhand
        pose = rpose
    
        hand_x = hand[:, 0*(len(LHAND_IDX)//3) : 1*(len(LHAND_IDX)//3)]
        hand_y = hand[:, 1*(len(LHAND_IDX)//3) : 2*(len(LHAND_IDX)//3)]
        hand_z = hand[:, 2*(len(LHAND_IDX)//3) : 3*(len(LHAND_IDX)//3)]
        hand = tf.concat([hand_x[..., tf.newaxis], hand_y[..., tf.newaxis], hand_z[..., tf.newaxis]], axis=-1)

        mean = tf.math.reduce_mean(hand, axis=1)[:, tf.newaxis, :]
        std = tf.math.reduce_std(hand, axis=1)[:, tf.newaxis, :]
        hand = (hand - mean) / std

        pose_x = pose[:, 0*(len(LPOSE_IDX)//3) : 1*(len(LPOSE_IDX)//3)]
        pose_y = pose[:, 1*(len(LPOSE_IDX)//3) : 2*(len(LPOSE_IDX)//3)]
        pose_z = pose[:, 2*(len(LPOSE_IDX)//3) : 3*(len(LPOSE_IDX)//3)]
        pose = tf.concat([pose_x[..., tf.newaxis], pose_y[..., tf.newaxis], pose_z[..., tf.newaxis]], axis=-1)
    
    x = tf.concat([hand, pose], axis=1)
    x = resize_pad(x)
    
    x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
    x = tf.reshape(x, (FRAME_LEN, len(LHAND_IDX) + len(LPOSE_IDX)))
    return x

table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=list(signs_to_idx.keys()),
        values=list(signs_to_idx.values()),
    ),
    default_value=tf.constant(-1),
    name="labels"
)

def preprocess_phrase(phrase):
    return table.lookup(phrase)

def convert_fn(landmarks, phrase):
    return pre_process(landmarks), preprocess_phrase(phrase)

In [137]:
batch_size = 256
train_len = int(0.8 * len(tf_records))

train_ds = tf.data.TFRecordDataset(tf_records[:train_len]).map(decode_fn).map(convert_fn).shuffle(buffer_size=batch_size).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE).cache()
valid_ds = tf.data.TFRecordDataset(tf_records[train_len:]).map(decode_fn).map(convert_fn).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE).cache()

In [138]:
# Define the number of data points you want to print
n = 1

# Create an iterator for the train and valid datasets
train_iterator = iter(train_ds)
valid_iterator = iter(valid_ds)

# Print data points from the training dataset
print("Training Data:")
for _ in range(n):
    try:
        landmarks, phrase = next(train_iterator)
        print("Landmarks:")
        print(type(landmarks))
        print(landmarks.shape)
        print(landmarks.numpy())
        print("Phrase:")
        print(phrase.shape)
        print(phrase)
        print("-" * 40)
    except StopIteration:
        break

Training Data:
Landmarks:
<class 'tensorflow.python.framework.ops.EagerTensor'>
(256, 128, 78)
[[[ 4.472087    0.         -1.2294794  ...  0.3156756   0.3845882
   -0.07328478]
  [ 4.4720936   0.         -1.1936464  ...  0.31524646  0.38171864
   -0.07025498]
  [ 4.472095    0.         -1.175357   ...  0.31442472  0.37426096
   -0.0721297 ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 4.472085   -2.546416   -1.2856165  ...  0.40856695  0.3860811
   -0.06707925]
  [ 4.4720907  -2.5079467  -1.2882923  ...  0.40897042  0.3814715
   -0.06725361]
  [ 4.472096   -2.5336735  -1.2802943  ...  0.4070918   0.3785196
   -0.06726068]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.    

# Creating the Model

In [69]:
class LandmarkEmbedding(layers.Layer):
    def __init__(self, num_hid=64, maxlen=100):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv2 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv3 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return self.conv3(x)

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class Transformer(keras.Model):
    def __init__(
        self,
        num_hid,
        num_head,
        num_feed_forward,
        source_maxlen,
        num_layers_enc,
        num_classes,
    ):
        super().__init__()
        self.loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.acc_metric = keras.metrics.Mean(name="accuracy")

        self.model = keras.Sequential(
            [
                layers.Input(shape=(source_maxlen, 1560)),
                LandmarkEmbedding(num_hid=num_hid, maxlen=source_maxlen)
            ] + 
            [
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ] + 
            [
                layers.Flatten(),
                layers.Dense(256, activation="relu"),
                layers.Dense(num_classes, activation="softmax")
            ]
        )

    def call(self, inputs, training):
        return self.model(inputs, training)

    @property
    def metrics(self):
        return [self.loss_metric]
    
    def train_step(self, batch):
        source = batch[0] # Source sequence
        target_class_labels = batch[1] # target label

        with tf.GradientTape() as tape:
            preds = self(source, training=True)
            loss = self.loss_function(target_class_labels, preds)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update accuracy metric
        accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(target_class_labels, preds))
        self.acc_metric.update_state(accuracy)

        # Update loss metric
        self.loss_metric.update_state(loss)

        return {"loss": self.loss_metric.result(), "accuracy": self.acc_metric.result()}

    def predict_class(self, input_sequence):
        input_sequence = tf.convert_to_tensor(input_sequence)
        predictions = self(input_sequence, training=False)
        predicted_class = tf.argmax(predictions, axis=-1).numpy()
        return predicted_class
    
    def predict_prob(self, input_sequence):
        input_sequence = tf.convert_to_tensor(input_sequence)
        predictions = self(input_sequence, training=False)
        return predictions

In [70]:
# model = Transformer(
#         num_hid=32,
#         num_head=2,
#         num_feed_forward=32,
#         source_maxlen=FRAME_LEN,
#         num_layers_enc=1,
#         num_classes=len(signs_to_idx),
# )

# optimizer = keras.optimizers.Adam(0.0001)
# model.compile(optimizer="adam")

In [144]:
num_hid=32
num_head=2
num_feed_forward=32
source_maxlen=FRAME_LEN
num_layers_enc=1
num_classes=len(signs_to_idx)

model = keras.Sequential(
            [
                layers.Input(shape=(source_maxlen, 78)),
                LandmarkEmbedding(num_hid=num_hid, maxlen=source_maxlen)
            ] + 
            [
                layers.LSTM(512, return_sequences=True),
                layers.Dropout(0.1),
                layers.LSTM(256, return_sequences=True),
                layers.Dropout(0.1),
                layers.LSTM(128),
            ] + 
            [
                layers.Flatten(),
                layers.Dense(256, activation="relu"),
                layers.Dense(num_classes, activation="softmax")
            ]
        )

# model = keras.Sequential(
#             [
#                 layers.Input(shape=(FRAME_LEN, 1560)),
#                 layers.Flatten(),
#                 layers.Dense(256, activation="relu"),
#                 layers.Dense(256, activation="relu"),
#                 layers.Dense(256, activation="relu"),
#                 layers.Dense(num_classes, activation="softmax")
#             ]
#         )

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['sparse_categorical_accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=4)])
model.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 landmark_embedding_20 (Land  (None, 16, 32)           50080     
 markEmbedding)                                                  
                                                                 
 lstm_42 (LSTM)              (None, 16, 512)           1116160   
                                                                 
 dropout_20 (Dropout)        (None, 16, 512)           0         
                                                                 
 lstm_43 (LSTM)              (None, 16, 256)           787456    
                                                                 
 dropout_21 (Dropout)        (None, 16, 256)           0         
                                                                 
 lstm_44 (LSTM)              (None, 128)               197120    
                                                     

In [145]:
history = model.fit(train_ds.take(10), epochs=10)

Epoch 1/10
10/10 [==============================] - 27s 2s/step - loss: 11.7010 - sparse_categorical_accuracy: 0.0047 - sparse_top_k_categorical_accuracy: 0.0195
Epoch 2/10
10/10 [==============================] - 23s 2s/step - loss: 12.5493 - sparse_categorical_accuracy: 0.0039 - sparse_top_k_categorical_accuracy: 0.0203
Epoch 3/10
 5/10 [==============>...............] - ETA: 9s - loss: 12.9893 - sparse_categorical_accuracy: 0.0070 - sparse_top_k_categorical_accuracy: 0.0219 

KeyboardInterrupt: 

In [119]:
x, y = next(iter(train_ds))
x

<tf.Tensor: shape=(1000, 100, 1560), dtype=float32, numpy=
array([[[ 3.60000000e+01,  4.58965153e-01,  4.56893891e-01, ...,
         -8.85483921e-02, -1.25149012e-01, -1.60655886e-01],
        [ 3.70000000e+01,  4.64591652e-01,  4.58463788e-01, ...,
         -5.40000945e-02, -8.04741085e-02, -1.06262930e-01],
        [ 3.80000000e+01,  4.67253625e-01,  4.61370945e-01, ...,
         -6.13685176e-02, -9.23710689e-02, -1.21201053e-01],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 8.00000000e+01,  4.51419383e-01,  4.50603396e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 8.10000000e+01,  4.52553242e-01,  

In [120]:
model.predict(x)

32/32 [==============================] - 4s 120ms/step


array([[0.00494485, 0.00405545, 0.00348067, ..., 0.00346177, 0.00294832,
        0.00302518],
       [0.00604455, 0.00393579, 0.00302099, ..., 0.00287214, 0.00299053,
        0.00334682],
       [0.00495973, 0.0037113 , 0.0039265 , ..., 0.0036895 , 0.00375038,
        0.00362635],
       ...,
       [0.00494575, 0.00376978, 0.00302454, ..., 0.00333682, 0.00273252,
        0.0029244 ],
       [0.00674426, 0.00459537, 0.00304566, ..., 0.0010017 , 0.00093502,
        0.00294182],
       [0.00451462, 0.00401646, 0.00361408, ..., 0.00324784, 0.00314015,
        0.00292332]], dtype=float32)

In [97]:
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dense(num_classes, activation="softmax")(x)
x

<tf.Tensor: shape=(1, 250), dtype=float32, numpy=
array([[0.00857084, 0.00559969, 0.00123946, 0.00135504, 0.00227907,
        0.00490996, 0.00246722, 0.00232943, 0.0091911 , 0.0038737 ,
        0.0052491 , 0.00666741, 0.00175896, 0.00340746, 0.00260326,
        0.00233817, 0.00359503, 0.00253037, 0.0031861 , 0.00261878,
        0.00218077, 0.0044235 , 0.00331729, 0.00251819, 0.00763835,
        0.00193001, 0.00471289, 0.00229954, 0.00195854, 0.00410927,
        0.00359797, 0.00206058, 0.00693681, 0.00192331, 0.00526182,
        0.00406014, 0.00467854, 0.00346762, 0.00377615, 0.00276598,
        0.00765121, 0.00573563, 0.00244224, 0.00566593, 0.00496827,
        0.00492922, 0.00454636, 0.01313426, 0.00133716, 0.00167505,
        0.00399361, 0.00427686, 0.00352446, 0.00269753, 0.00483882,
        0.00361537, 0.00333719, 0.00242267, 0.00714315, 0.00244263,
        0.00127311, 0.0057013 , 0.00474184, 0.00511408, 0.00745919,
        0.0062921 , 0.00326644, 0.00927508, 0.00393929, 0.00357656

   7129/Unknown - 910s 127ms/step - loss: 5.6700 - sparse_categorical_accuracy: 0.0042 - sparse_top_k_categorical_accuracy: 0.0168

KeyboardInterrupt: 

In [51]:
x, y = next(iter(train_ds))
model.predict(x)

2/2 [==============================] - 0s 7ms/step


array([[0.00294294, 0.00228894, 0.00352863, ..., 0.0034796 , 0.00197209,
        0.00483718],
       [0.00442709, 0.0024515 , 0.00243251, ..., 0.00400932, 0.00407085,
        0.00424809],
       [0.00381204, 0.00196817, 0.00378237, ..., 0.0028785 , 0.00249932,
        0.00407746],
       ...,
       [0.0031885 , 0.00196957, 0.00369545, ..., 0.00341888, 0.0020477 ,
        0.00447635],
       [0.00400573, 0.00340239, 0.00494421, ..., 0.00377446, 0.00335186,
        0.00371072],
       [0.00424815, 0.00428819, 0.00343494, ..., 0.00384066, 0.00378869,
        0.00504714]], dtype=float32)

In [50]:
y

<tf.Tensor: shape=(64,), dtype=int32, numpy=
array([231, 108,  81,  80,  23,  19, 220,  19,  89,  45,  48,  34, 190,
        24, 181,  93,  49,   6, 134, 207,  86,  74, 243,  67, 159, 195,
        13, 144, 156,  78, 233, 235, 233, 196, 198,  48,  35, 121,  47,
       140, 192, 164, 165,  25,  66, 143,  62,  32,  12,  97, 243, 172,
       102, 165, 127,   0,  65, 239, 215,  25,  76,  77, 180, 157])>

In [122]:
batch = next(iter(valid_ds))
pred = model.predict_class(batch[0])

In [123]:
pred

array([132,  35, 134, 134,  92, 133, 139, 133,  92,   4, 189, 133, 134,
       134, 134,   4,   4, 132, 134,  47, 134, 134, 134, 189, 133, 133,
       132, 134, 132, 132,  47, 132, 133, 133, 134, 134, 132,  92, 134,
       134,  47, 133, 134,  92, 133, 132, 133, 133, 189, 132,  92,  92,
       134, 133,   4,  92, 134, 133, 134, 134, 134, 133, 133,   4],
      dtype=int64)

In [124]:
probs = model.predict_prob(batch[0])
max(probs[0].numpy())

0.011367946

In [37]:
model.summary()

Model: "transformer_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 250)               704250    
                                                                 
Total params: 704,254
Trainable params: 704,250
Non-trainable params: 4
_________________________________________________________________


In [33]:
# Transformer variables are customized from original keras tutorial to suit this dataset.
# Reference: https://www.kaggle.com/code/shlomoron/aslfr-a-simple-transformer/notebook

# batch = next(iter(valid_ds))

# # The vocabulary to convert predicted indices into characters
# idx_to_char = list(char_to_num.keys())
# display_cb = DisplayOutputs(
#     batch, idx_to_char, target_start_token_idx=char_to_num['<'], target_end_token_idx=char_to_num['>']
# )  # set the arguments as per vocabulary index for '<' and '>'


# loss_fn = tf.keras.losses.CategoricalCrossentropy(
#     from_logits=True, label_smoothing=0.1,
# )




In [337]:
batches = [batch for batch in valid_ds]

preds_list = []
ground_truth_list = []

for batch in batches[:1]:
    source = batch[0]
    target = batch[1].numpy()
    bs = tf.shape(source)[0]
    preds = model.generate(source, start_token_idx)
    preds = preds.numpy()

    for i in range(bs):
        target_text = "".join([idx_to_char[_] for _ in target[i, :]])
        ground_truth_list.append(target_text.replace('P', ''))
        prediction = ""
        for idx in preds[i, :]:
            prediction += idx_to_char[idx]
            if idx == end_token_idx:
                break
        preds_list.append(prediction)

for i in range(30):
    print(ground_truth_list[i])
    print(preds_list[i])
    print('\n~~~\n')

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [8,1560], [batch]: [5,1560] [Op:IteratorGetNext]